In [56]:
import pandas as pd
import random
from collections import Counter 
import operator
import math
import mmh3
import numpy as np

In [17]:
# Select the dataset
# data_file = 'capture20110815-2.pcap.netflow.labeled' # dataset 43
# data_file = 'capture20110811.pcap.netflow.labeled' # dataset 46
# data_file = 'capture20110817.pcap.netflow.labeled' # dataset 49
data_file = 'capture20110817.pcap.netflow.labeled' #dataset 50

In [28]:
infected_host = '147.32.84.165'
columns=['Date','Duration','Protocol','SrcIP', 'srcPort','DstIP', 'DstPort', 'Flags', 'TOS', 'Packets', 'Bytes', 'Label']
lst=[]
with open(data_file) as fp:
    for cnt, line in enumerate(fp):
        if cnt!=0:
            data=line.split("\t")
            data = list(filter(None, data)) 
            date = data[0]
            duration = data[1]
            protocol = data[2]
            src = data[3].split(':')
            if len(src) < 2: # Set port to 0 if no port provided
                src.append(0)
            dst = data[5].split(':')
            if len(dst) < 2:
                dst.append(0)
            if (src[0] != infected_host and dst[0] != infected_host):
                continue  # skip rows that are not about the infected host to save memory
            if (src[0] == dst[0]):
                print("CONNECT TO SELF")
            flags = data[6]
            tos = data[7]
            packets = data[8]
            byteno = data[9]
            flows = data[10]
            label = data[11].rstrip()
            lst.append([date, duration, protocol, src[0], src[1], dst[0], dst[1], flags, tos, packets, byteno, label])
df=pd.DataFrame(lst, columns=columns)
df.head()

,Date,Duration,Protocol,SrcIP,srcPort,DstIP,DstPort,Flags,TOS,Packets,Bytes,Label
0,2011-08-17 12:14:50.903,2.993,TCP,147.32.96.45,2048,147.32.84.165,21,S_,0,2,148,Botnet
1,2011-08-17 12:14:59.896,0.000,TCP,147.32.96.45,2048,147.32.84.165,21,S_,0,1,74,Botnet
2,2011-08-17 12:26:44.161,2.994,TCP,147.32.96.45,2048,147.32.84.165,21,S_,0,2,148,Botnet
3,2011-08-17 12:26:53.153,0.000,TCP,147.32.96.45,2048,147.32.84.165,21,S_,0,1,74,Botnet
4,2011-08-17 12:28:41.718,0.000,UDP,147.32.84.165,1025,147.32.80.9,53,INT,0,1,64,Botnet


In [19]:
# infected_host = '147.32.84.165'
# df_infected=df.loc[(df['SrcIP']==infected_host) | (df['DstIP']==infected_host)]
# df_infected=df_infected.reset_index()
df_infected = df

In [14]:
df_infected.to_pickle(data_file + '_pickled.pkl')

In [ ]:
df_infected = pd.read_pickle(data_file + '_pickled.pkl')
# df = pd.read_pickle('pickled_data.pkl')

In [79]:
# Count connections
ip_map = {}
ip_list = []
for index, row in df_infected.iterrows():
    if row["SrcIP"] == infected_host:
        ip_map.setdefault(row["DstIP"],0)
        ip_map[row["DstIP"]] += 1
        ip_list.append(row["DstIP"])
    else:
        ip_map.setdefault(row["SrcIP"],0)
        ip_map[row["SrcIP"]] += 1
        ip_list.append(row["SrcIP"])
        

In [48]:
# Show the 10 most frequestly connecting IPs 
print(Counter(ip_map).most_common(10))
assert len(df_infected) == sum(Counter(ip_map).values())
len(df_infected)

[('147.32.80.9', 30533), ('212.117.171.138', 1851), ('188.72.241.107', 1203), ('173.192.170.88', 857), ('204.12.208.59', 493), ('195.190.13.78', 477), ('74.125.39.27', 314), ('209.190.94.170', 310), ('83.133.119.197', 212), ('65.54.188.72', 183)]


48186

In [49]:
# Calculate the exact fequencies
most_freq_ips = Counter(ip_map).most_common(10)
for ip in Counter(ip_map).most_common(10):
    ips.append(ip[0])
    print(f"{ip[0]} : {(ip[1] / len(df_infected) * 100)}%")


147.32.80.9 : 63.364877765326035%
212.117.171.138 : 3.841364711742%
188.72.241.107 : 2.49657576889553%
173.192.170.88 : 1.7785248827460256%
204.12.208.59 : 1.0231187481841197%
195.190.13.78 : 0.9899140829286515%
74.125.39.27 : 0.6516415556385672%
209.190.94.170 : 0.6433403893247002%
83.133.119.197 : 0.4399618146349562%
65.54.188.72 : 0.37977835885941974%


In [29]:
def selectKItems(stream, n, k): 
    i=0
    reservoir = [0]*k
    for i in range(k): 
        if (df_infected.iloc[i, :]["DstIP"] == infected_host):
            reservoir[i] = df_infected.iloc[i, :]["SrcIP"]
        else:
            reservoir[i] = df_infected.iloc[i, :]["DstIP"]

    while(i < n): 
        j = random.randrange(i+1)
        
        if(j < k): 
            if (df_infected.iloc[i, :]["DstIP"] == infected_host):
                reservoir[j] = df_infected.iloc[i, :]["SrcIP"]
            else:
                reservoir[j] = df_infected.iloc[i, :]["DstIP"]
        i+=1

    return reservoir

In [74]:
reservoirSizes = [100, 250, 500, 1000, 5000, 10000, 25000, len(df_infected)]
for reservoir in reservoirSizes:
    print(f"\n\nReservoir size = {reservoir}")
    kfreqIPS = selectKItems(df_infected, len(df_infected), reservoir)
    for ip in Counter(kfreqIPS).most_common(10):
        print(f"{ip[0]} & {ip[1]} & {(ip[1] / len(kfreqIPS) * 100)}\\% \\\\ \\hline")



Reservoir size = 100
147.32.80.9 & 56 & 56.00000000000001\% \\ \hline
212.117.171.138 & 3 & 3.0\% \\ \hline
83.133.119.197 & 3 & 3.0\% \\ \hline
173.192.170.88 & 3 & 3.0\% \\ \hline
74.125.232.218 & 2 & 2.0\% \\ \hline
209.86.93.226 & 1 & 1.0\% \\ \hline
71.74.56.243 & 1 & 1.0\% \\ \hline
68.67.185.205 & 1 & 1.0\% \\ \hline
12.15.146.28 & 1 & 1.0\% \\ \hline
174.36.246.56 & 1 & 1.0\% \\ \hline


Reservoir size = 250
147.32.80.9 & 153 & 61.199999999999996\% \\ \hline
212.117.171.138 & 10 & 4.0\% \\ \hline
188.72.241.107 & 5 & 2.0\% \\ \hline
173.192.170.88 & 5 & 2.0\% \\ \hline
74.125.39.27 & 3 & 1.2\% \\ \hline
195.190.13.78 & 3 & 1.2\% \\ \hline
204.12.208.59 & 3 & 1.2\% \\ \hline
195.113.232.88 & 2 & 0.8\% \\ \hline
65.55.37.120 & 2 & 0.8\% \\ \hline
209.190.94.170 & 2 & 0.8\% \\ \hline


Reservoir size = 500
147.32.80.9 & 323 & 64.60000000000001\% \\ \hline
212.117.171.138 & 15 & 3.0\% \\ \hline
188.72.241.107 & 14 & 2.8000000000000003\% \\ \hline
173.192.170.88 & 10 & 2.0\% \\ \h

# MIN-WISE sketch


In [90]:
class CountMinSketch:
    def __init__(self, w, d):
        self.size = w*d
        self.w = w
        self.hash_count = d
        self.cm_array =  [[0]*w for i in range(d)]
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.w
            self.cm_array[seed][result] += 1
        
    def point(self, string):
        min = 9999999
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.w
            if self.cm_array[seed][result]<min:
                min = self.cm_array[seed][result]
        return min

# compute the frequnt ips for different values of d and w
ips = list(ip_map.keys())
for epsilon in [0.0001, 0.001, 0.005, 0.01, 0.1]:
    for delta in [0.0001, 0.001, 0.005, 0.01, 0.1]:
        
        # calculate the w, d
        w = round(math.e/epsilon)
        d = round(math.log(1/delta))
        count_min_matrix = CountMinSketch(int(w), int(d))

        for ip in ip_list:
                count_min_matrix.add(ip)
                
        # find frequencies
        count_min = {}
        for ip in ips:
            count_min[ip] = count_min_matrix.point(ip)

        sorted_count_min = sorted(count_min.items(), key=lambda kv: kv[1], reverse = True)

        difference_w_ground = 0
        for i in range(10):
            #find the difference between the count min and the ground truth
            difference_w_ground+= abs(sorted_count_min[i][1] - most_freq_ips[i][1])

        print(f'For w={w} and d={d} accuracy={ 100-(difference_w_ground*100.0/len(ip_list))} %')
        print('epsilon= ',epsilon, 'and delta=', delta)

For w=27183 and d=9 accuracy=100.0 %
epsilon=  0.0001 and delta= 0.0001
For w=27183 and d=7 accuracy=100.0 %
epsilon=  0.0001 and delta= 0.001
For w=27183 and d=5 accuracy=100.0 %
epsilon=  0.0001 and delta= 0.005
For w=27183 and d=5 accuracy=100.0 %
epsilon=  0.0001 and delta= 0.01
For w=27183 and d=2 accuracy=100.0 %
epsilon=  0.0001 and delta= 0.1
For w=2718 and d=9 accuracy=100.0 %
epsilon=  0.001 and delta= 0.0001
For w=2718 and d=7 accuracy=100.0 %
epsilon=  0.001 and delta= 0.001
For w=2718 and d=5 accuracy=100.0 %
epsilon=  0.001 and delta= 0.005
For w=2718 and d=5 accuracy=100.0 %
epsilon=  0.001 and delta= 0.01
For w=2718 and d=2 accuracy=99.97717179263687 %
epsilon=  0.001 and delta= 0.1
For w=544 and d=9 accuracy=99.93566596106753 %
epsilon=  0.005 and delta= 0.0001
For w=544 and d=7 accuracy=99.9252895031752 %
epsilon=  0.005 and delta= 0.001
For w=544 and d=5 accuracy=99.91076246212593 %
epsilon=  0.005 and delta= 0.005
For w=544 and d=5 accuracy=99.91076246212593 %
epsil